In [2]:
import os
import math
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

from utils import *

In [25]:
fontaines_gdf = gpd.read_file("fontaines.geojson")
parcs_gdf = gpd.read_file("parcs.geojson")
bikes_gdf = gpd.read_file("bike.geojson")

In [8]:
fontaines = fontaines_gdf[fontaines_gdf["nom_parc_lieu"]=="du Mont-Royal"]

In [9]:
parcs = parcs_gdf[parcs_gdf["objectid"]=="5629"]

In [19]:
from constraint import *

# Define the minimum distance between points
min_distance = 0.5

# Define the function to compute the Euclidean distance between two points
def distance(point1, point2):
    return ((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2 + (point1[2] - point2[2])**2)**0.5

# Define the problem
problem = Problem()

# Add variables for each point
for i in range(len(points)):
    problem.addVariables([f'x_{i}', f'y_{i}', f'z_{i}'], domain=(-1, 1))

# Add constraints to enforce minimum distance between each pair of points
for i in range(len(points)):
    for j in range(i+1, len(points)):
        problem.addConstraint(lambda x1, y1, z1, x2, y2, z2: distance((x1, y1, z1), (x2, y2, z2)) >= min_distance, 
                              [f'x_{i}', f'y_{i}', f'z_{i}', f'x_{j}', f'y_{j}', f'z_{j}'])

# Solve the problem
solutions = problem.getSolutions()

# Get the best solution
best_solution = min(solutions, key=lambda x: sum([(x[f'x_{i}'] - points[i][0])**2 + (x[f'y_{i}'] - points[i][1])**2 + (x[f'z_{i}'] - points[i][2])**2 for i in range(len(points))]))

# Print the result
points_adjusted = [(best_solution[f'x_{i}'], best_solution[f'y_{i}'], best_solution[f'z_{i}']) for i in range(len(points))]
print(points_adjusted)


[(1, 1, 1), (1, -1, -1), (1, 1, -1), (1, -1, 1)]


In [23]:
from shapely.geometry import Point, LineString
from geopy.distance import geodesic

# Coordonnées du point de référence
point = Point(-73.935242, 40.730610)

# Coordonnées du linestring
linestring = LineString([
    (-73.983512, 40.760282),
    (-73.975588, 40.756057),
    (-73.966392, 40.757947),
    (-73.955790, 40.767303),
    (-73.943412, 40.768076)
])

# Trouver le point le plus proche du linestring
nearest_point = linestring.interpolate(linestring.project(point))

# Calculer la distance Haversine entre le point et le point le plus proche
distance = geodesic((point.y, point.x), (nearest_point.y, nearest_point.x)).km

# Afficher la distance résultante
print(distance)


4.217389047629202


In [24]:
from shapely.geometry import LineString, Polygon

# Coordonnées du linestring
linestring = LineString([
    (-73.983512, 40.760282),
    (-73.975588, 40.756057),
    (-73.966392, 40.757947),
    (-73.955790, 40.767303),
    (-73.943412, 40.768076),
    (-73.935242, 40.730610),
    (-73.927073, 40.693144)
])

# Coordonnées du polygon
polygon = Polygon([
    (-73.977851, 40.763338),
    (-73.978729, 40.752792),
    (-73.962345, 40.748822),
    (-73.959255, 40.755845),
    (-73.960618, 40.757829),
    (-73.961981, 40.760825),
    (-73.965385, 40.763482)
])

# Extraire la partie du linestring qui se trouve dans le polygon
intersection = linestring.intersection(polygon)

# Afficher le résultat
print(intersection)


LINESTRING (-73.97833522429573 40.75752179336818, -73.975588 40.756057, -73.966392 40.757947, -73.96259109055482 40.76130120758055)
